<a href="https://colab.research.google.com/github/JoeRishon/DLNN-Resources/blob/master/RNNText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import numpy as np
import random

In [3]:
import io
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print(len(text))

606208/600901 [==============================] - 0s 0us/step
600893


assign characters to a numerical value. This makes it easier for the model to predict which character comes next.

In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


Next, we create multidimensional arrays out of the text data that we inputed so that the entire corpus of text is enumerated (is assigned a number instead of a character). This will be used for training and makes everything more coherent.

In [5]:
sentences = []
next_chars = []
for i in range(0, len(text) - 40, 3):
    sentences.append(text[i: i + 40])
    next_chars.append(text[i + 40])

x = np.zeros((len(sentences), 40, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

let's build the model with an LSTM layer

In [6]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(40, len(chars))))
model.add(keras.layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

This function will compare the sample predictions with the original text. The temperature represents how accurate or confident the model's predictions will be.

In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Next, train the model. It is recommended that at least 20 epochs are given to the model, however this can take quite a while so be prepared. The output text should be around 400 characters.

In [9]:
model.fit(x, y, batch_size=1, epochs=1)
start_index = random.randint(0, len(text) - 40 - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    generated = ''
    sentence = text[start_index: start_index + 40]
    generated += sentence

    for i in range(400):
        x_pred = np.zeros((1, 40, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
with open('example.txt', 'w') as f:
    f.write(generated)

  2952/200285 [..............................] - ETA: 41:33 - loss: 2.7575

KeyboardInterrupt: ignored

Finally, save the sample text to your computer (optional)

In [ ]:
from google.colab import files
files.download('example.txt')